In [ ]:
import skimpy

In [ ]:
x = skimpy.IntArray(10, 1)

In [ ]:
x

In [ ]:
%%time 
x[2] = 3

In [ ]:
x

In [ ]:
%%time
x[3:6] = 2

In [ ]:
x

In [ ]:
%%time
2 * x

In [ ]:
%%time
y = 2 * x

In [ ]:
y

In [ ]:
x

In [ ]:
%%time
((x + y)**x + 56 * x - 3 * x) // x**2

In [ ]:
big = skimpy.IntArray(1024 * 1024, 1)

In [ ]:
big

In [ ]:
len(big)

In [ ]:
%%time
for i in range(1024):
    big[i] = i

In [ ]:
big

In [ ]:
%%time
for i in range(1024):
    big[1024 * i: 1024 * (i + 1)] = big[:1024]

In [ ]:
%%time
big * big * big * big

In [ ]:
import numpy as np

In [ ]:
x = np.random.rand(1024 * 1024)

In [ ]:
%%time
z = x * x * x * x